# INSERT INTO DB

In [1]:
import pandas as pd
import numpy as np

import warnings
import os
warnings.filterwarnings(action='ignore')

In [ ]:
items = pd.read_json('./items_prc.json')

In [22]:
reviews = pd.read_json('./reviews_prc_2.json')

In [23]:
reviews = reviews.replace({np.nan: None})

In [22]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  860 non-null    int64 
 1   cat      860 non-null    object
 2   name     860 non-null    object
 3   price    860 non-null    int64 
 4   img      860 non-null    object
 5   link     860 non-null    object
dtypes: int64(2), object(4)
memory usage: 40.4+ KB


In [18]:
import sqlite3
conn = sqlite3.connect('./apparel.db')
cur = conn.cursor()

### item table

In [8]:
# 데이터 입력에 필요한 형태로 변환
items = items[['item_id', 'cat', 'name', 'price', 'img', 'link']]
items.item_id = items.item_id.astype(str)

In [9]:
for row in items.itertuples(index=False, name=None):
    cur.execute('''
                INSERT INTO item VALUES (?, ?, ?, ?, ?, ?); ''', row)
    conn.commit()

## reviews => user, review로 나누기

In [24]:
reviews.head()

,품번,아이디,별점,키,몸무게,평소사이즈,리뷰,사이즈,색상
0,464,mg,5,161~165,None,None,아직 안입어봤지만 여름에 입기 좋을거같아요 기본 디자인에 블랙앤화이트라 무난하고 루...,F,블랙
1,464,188592267,5,None,None,None,후기 두께도 얇은편이고 니트느낌나는 재질이기도 하고 사이즈도 널럴해서 바람도 잘 통...,F,블랙
2,464,363732161,5,156~160,51 ~ 55,2.0,완전 제서타일!! 빨앗는데 달라지는 느낌 없구 됴아요!! 가오리 핏 됴아용,F,블랙
3,464,291515456,5,156~160,65~,4.0,넉넉하고 편함 얆은 니트라 덥지도않아요 컬러감좋아요.딸아이 입고 이쁘다맘에들어해요,F,크림
4,464,225725978,1,None,None,None,옷은 너무 맘에 들어요 근데..딱 두번 입었는데 폈어요ㅠㅠ 옷 험하게 입는 스타일도...,F,크림


In [25]:
# user, review 정보 나누기
user = reviews[['아이디', '키', '몸무게', '평소사이즈']]
review = reviews[['아이디', '품번', '별점', '리뷰', '사이즈', '색상']]

### user Table

In [26]:
for col in ['키', '몸무게', '평소사이즈']:
    print(col)
    print(user[col].unique())

키
['161~165' None '156~160' '166~170' '151~155' '171~175' '139이하' '176이상'
 '146~150']
몸무게
[None '51 ~ 55' '65~' '61 ~ 65' '56 ~ 60' '46 ~ 50' '~45']
평소사이즈
[None 2.0 4.0 3.0 5.0 1.0]


In [27]:
user_dup = user.loc[user['아이디'].duplicated()].index
user.drop(user_dup, axis=0, inplace=True)
user.reset_index(drop=True, inplace=True)
user.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880 entries, 0 to 1879
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   아이디     1880 non-null   object
 1   키       1396 non-null   object
 2   몸무게     1305 non-null   object
 3   평소사이즈   1282 non-null   object
dtypes: object(4)
memory usage: 58.9+ KB


In [28]:
# 아이디 중복 확인 후 데이터 입력
for row in user.itertuples(name=None, index=False):
    # cur.execute(f'''
    #             INSERT INTO user VALUES(?, ?, ?, ?)
    #             SELECT * FROM user
    #             WHERE NOT EXISTS
    #             (SELECT 1 FROM USER
    #             WHERE id == {row[0]});''', row)
    cur.execute('''
                INSERT OR IGNORE
                INTO user VALUES(?,?,?,?);''', row) # 아이디 중복 제외
    
    conn.commit()

In [29]:
cur.execute('''
            SELECT COUNT(*) FROM user;''')
print(cur.fetchone())

(40847,)


### review Table

In [30]:
review = review[['아이디', '품번', '별점', '리뷰', '사이즈', '색상']]
review_dup = review.loc[review['리뷰'].duplicated()==True].index
review.drop(review_dup, axis=0, inplace=True)
review.reset_index(drop=True, inplace=True)
len(review)

3058

In [31]:
review.duplicated().sum()

0

In [15]:
# '''
# review table 줄바꿈 제거 필요
# '''
# cur.execute('''
#             UPDATE review
#             SET comment = replace(comment, '\n', ' ');''')
# conn.commit()

In [32]:
for row in review.itertuples(name=None, index=True):
    cur.execute('''
                INSERT OR IGNORE
                INTO REVIEW VALUES(?,?,?,?,?,?,?);''', row)
    conn.commit()

## 연결 해제

In [33]:
cur.close()
conn.close()